In [1]:
!pip install ktrain

     |████████████████████████████████| 25.3 MB 39.9 MB/s 
     |████████████████████████████████| 6.8 MB 40.5 MB/s 
     |████████████████████████████████| 981 kB 26.4 MB/s 
     |████████████████████████████████| 263 kB 39.4 MB/s 
     |████████████████████████████████| 2.8 MB 33.8 MB/s 
     |████████████████████████████████| 1.2 MB 34.6 MB/s 
     |████████████████████████████████| 468 kB 39.4 MB/s 
     |████████████████████████████████| 596 kB 31.8 MB/s 
     |████████████████████████████████| 895 kB 27.6 MB/s 
     |████████████████████████████████| 3.3 MB 25.1 MB/s 
     |████████████████████████████████| 61 kB 475 kB/s 
  Created wheel for ktrain: filename=ktrain-0.28.3-py3-none-any.whl size=25292659 sha256=5515fbc5cb54f9fe05f99864695a5b7da7cf6f7ed6c5f6506be03df49dfdd7db
  Stored in directory: /root/.cache/pip/wheels/6a/7e/c3/f46cdfc2b81c54424923b1405d7e670c35cacc11ada9a47b1c
  Created wheel for seqeval: filename=seqeval-0.0.19-py3-none-any.whl size=9929 sha256=0005da418602dd8

In [2]:
!wget https://github.com/mayanksinghkgp/Model_Deployment/raw/main/data/test.xlsx
!wget https://github.com/mayanksinghkgp/Model_Deployment/raw/main/data/train.xlsx

--2021-12-19 17:37:31--  https://github.com/mayanksinghkgp/Model_Deployment/raw/main/data/test.xlsx
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/mayanksinghkgp/Model_Deployment/main/data/test.xlsx [following]
--2021-12-19 17:37:32--  https://raw.githubusercontent.com/mayanksinghkgp/Model_Deployment/main/data/test.xlsx
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13721650 (13M) [application/octet-stream]
Saving to: ‘test.xlsx’

test.xlsx           100%[===================>]  13.09M  --.-KB/s    in 0.1s    

2021-12-19 17:37:33 (90.8 MB/s) - ‘test.xlsx’ saved [13721650/13721650]

--2021-12-1

In [3]:
import pandas as pd
import numpy as np
import ktrain
from ktrain import text
import tensorflow as tf

In [4]:
data_test = pd.read_excel('/content/test.xlsx', dtype= str)
data_train = pd.read_excel('/content/train.xlsx', dtype = str)

In [7]:
data_train.tail(5)

,Reviews,Sentiment
24995,Everyone plays their part pretty well in this ...,pos
24996,It happened with Assault on Prescient 13 in 20...,neg
24997,My God. This movie was awful. I can't complain...,neg
24998,"When I first popped in Happy Birthday to Me, I...",neg
24999,"So why does this show suck? Unfortunately, tha...",neg


In [6]:
text.print_text_classifiers()

fasttext: a fastText-like model [http://arxiv.org/pdf/1607.01759.pdf]
logreg: logistic regression using a trainable Embedding layer
nbsvm: NBSVM model [http://www.aclweb.org/anthology/P12-2018]
bigru: Bidirectional GRU with pretrained fasttext word vectors [https://fasttext.cc/docs/en/crawl-vectors.html]
standard_gru: simple 2-layer GRU with randomly initialized embeddings
bert: Bidirectional Encoder Representations from Transformers (BERT) from keras_bert [https://arxiv.org/abs/1810.04805]
distilbert: distilled, smaller, and faster BERT from Hugging Face transformers [https://arxiv.org/abs/1910.01108]


In [8]:
(train, val, preproc) = text.texts_from_df(train_df=data_train, text_column='Reviews', label_columns='Sentiment',
                   val_df = data_test,
                   maxlen = 400,
                   preprocess_mode = 'distilbert')

['neg', 'pos']
   neg  pos
0  1.0  0.0
1  1.0  0.0
2  1.0  0.0
3  1.0  0.0
4  1.0  0.0
['neg', 'pos']
   neg  pos
0  0.0  1.0
1  0.0  1.0
2  1.0  0.0
3  0.0  1.0
4  1.0  0.0


Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

preprocessing train...
language: en
train sequence lengths:
	mean : 234
	95percentile : 598
	99percentile : 913


Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 234
	95percentile : 598
	99percentile : 913


In [9]:
model = text.text_classifier(name = 'distilbert', train_data = train, preproc=preproc)

Is Multi-Label? False
maxlen is 400


Downloading:   0%|          | 0.00/363M [00:00<?, ?B/s]

done.


In [10]:
learner = ktrain.get_learner(model = model,
                             train_data = train,
                             val_data = val,
                             batch_size = 6)

In [ ]:
learner.fit_onecycle(lr = 2e-5, epochs=2)



begin training using onecycle policy with max lr of 2e-05...
Epoch 1/2
 711/4167 [====>.........................] - ETA: 32:46 - loss: 0.3459 - accuracy: 0.8713

In [ ]:
predictor = ktrain.get_predictor(learner.model, preproc)

In [ ]:
predictor.save('/content/drive/MyDrive/Colab_Files/distillbert')

In [ ]:
data = ["Despite the insane loneliness and high stakes of being stuck alone on Mars, The Martian's witty script keeps humor and excitement alive in every scene. Space may be dangerous and scary, but the joy of scientific discovery is intoxicating",
        "While he's fantastic whenever he's on the move, using his cool stoicism to counteract the rampaging bus, Keanu Reeves can't quite match his costar in the quiet moments of Speed, which falter under his expressionless gaze.", 'another great movie. must watch to everyone']

In [ ]:
predictor.predict(data)

In [ ]:
predictor.get_classes()

In [ ]:
predictor.predict(data, return_proba=True)